In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import sqlite3

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import sqlite3

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [8]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [9]:
Measurement.__table__

Table('measurement', MetaData(bind=None), Column('id', INTEGER(), table=<measurement>, primary_key=True, nullable=False), Column('station', TEXT(), table=<measurement>), Column('date', TEXT(), table=<measurement>), Column('prcp', FLOAT(), table=<measurement>), Column('tobs', FLOAT(), table=<measurement>), schema=None)

In [10]:
Station.__table__

Table('station', MetaData(bind=None), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', TEXT(), table=<station>), Column('name', TEXT(), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('elevation', FLOAT(), table=<station>), schema=None)

In [11]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [12]:
#First explore the data in terms of date range in the database
all = pd.DataFrame(session.query(Measurement.date, Measurement.prcp, Measurement.tobs, Measurement.station).all())
all
#date ramge for all years: 2010-2017 (June) and 2010-2016 (Dec)

,date,prcp,tobs,station
0,2010-01-01,0.08,65.0,USC00519397
1,2010-01-02,0.00,63.0,USC00519397
2,2010-01-03,0.00,74.0,USC00519397
3,2010-01-04,0.00,76.0,USC00519397
4,2010-01-06,NaN,73.0,USC00519397
...,...,...,...,...
19545,2017-08-19,0.09,71.0,USC00516128
19546,2017-08-20,NaN,78.0,USC00516128
19547,2017-08-21,0.56,76.0,USC00516128
19548,2017-08-22,0.50,76.0,USC00516128


In [13]:
# Identify key statistical data in June across all of the stations and years
def get_date(y,m,d):
    d = dt.date(y,m,d)
    return d

def month_query(date1, date2):
    res = session.query(Measurement.date, Measurement.prcp, Measurement.station, 
                  Measurement.tobs).filter(Measurement.date>=date1, 
                  Measurement.date<=date2).all()
    return res
june_2010 = month_query(get_date(2010,6,1), get_date(2010, 6, 30))
june_2011 = month_query(get_date(2011,6,1), get_date(2011,6,30))
june_2012 = month_query(get_date(2012,6,1), get_date(2012,6,30))
june_2013 = month_query(get_date(2013,6,1), get_date(2013,6,30))
june_2014 = month_query(get_date(2014,6,1), get_date(2014,6,30))
june_2015 = month_query(get_date(2015,6,1), get_date(2015,6,30))
june_2016 = month_query(get_date(2016,6,1), get_date(2016,6,30))
june_2017 = month_query(get_date(2017,6,1), get_date(2017,6,30))

def month_dataframe(j):
    dataframe = pd.DataFrame(j,columns = ["date", "precipitation", "station", "tobs"] )
    return dataframe
june_2010df = month_dataframe(june_2010)
june_2011df = month_dataframe(june_2011)
june_2012df = month_dataframe(june_2012)
june_2013df = month_dataframe(june_2013)
june_2014df = month_dataframe(june_2014)
june_2015df = month_dataframe(june_2015)
june_2016df = month_dataframe(june_2016)
june_2017df = month_dataframe(june_2017)

frames = (june_2010df, june_2011df, june_2012df, june_2013df,
                 june_2014df, june_2015df, june_2016df, june_2017df)
junedf = pd.concat(frames)
junedf.sort_values("date", inplace = True)
junedf.groupby("date").sum()


,precipitation,tobs
date,,
2010-06-01,0.15,662.0
2010-06-02,0.07,671.0
2010-06-03,0.24,670.0
2010-06-04,0.01,656.0
2010-06-05,0.09,608.0
...,...,...
2017-06-26,0.10,553.0
2017-06-27,0.11,468.0
2017-06-28,0.03,466.0


In [14]:
# Identify key statistical data in June across all of the stations and years
junedf.describe()

,precipitation,tobs
count,1574.000000,1700.000000
mean,0.136360,74.944118
std,0.335731,3.257417
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [15]:
# Identify key statistical data in Dec across all of the stations and years


dec_2010 = month_query(get_date(2010,12,1), get_date(2010, 12, 31))
dec_2011 = month_query(get_date(2011,12,1), get_date(2011,12,31))
dec_2012 = month_query(get_date(2012,12,1), get_date(2012,12,31))
dec_2013 = month_query(get_date(2013,12,1), get_date(2013,12,31))
dec_2014 = month_query(get_date(2014,12,1), get_date(2014,12,31))
dec_2015 = month_query(get_date(2015,12,1), get_date(2015,12,31))
dec_2016 = month_query(get_date(2016,12,1), get_date(2016,12,31))

dec_2010df = month_dataframe(dec_2010)
dec_2011df = month_dataframe(dec_2011)
dec_2012df = month_dataframe(dec_2012)
dec_2013df = month_dataframe(dec_2013)
dec_2014df = month_dataframe(dec_2014)
dec_2015df = month_dataframe(dec_2015)
dec_2016df = month_dataframe(dec_2016)


frames12 = (dec_2010df, dec_2011df, dec_2012df, dec_2013df,
                 dec_2014df, dec_2015df, dec_2016df)
decdf = pd.concat(frames12)
decdf.sort_values("date", inplace = True)
decdf.groupby("date").sum()


,precipitation,tobs
date,,
2010-12-01,4.52,585.0
2010-12-02,0.81,580.0
2010-12-03,3.24,575.0
2010-12-04,0.19,524.0
2010-12-05,0.00,499.0
...,...,...
2016-12-27,0.24,438.0
2016-12-28,0.41,501.0
2016-12-29,2.38,509.0


In [16]:
decdf.describe()

,precipitation,tobs
count,1405.000000,1517.000000
mean,0.216819,71.041529
std,0.541399,3.745920
min,0.000000,56.000000
25%,0.000000,69.000000
50%,0.030000,71.000000
75%,0.150000,74.000000
max,6.420000,83.000000


# Challenge

In [19]:
#Repeat the exercise for 2016-2017 only - June 
june_2016 = month_query(get_date(2016,6,1), get_date(2016,6,30))
june_2017 = month_query(get_date(2017,6,1), get_date(2017,6,30))
june_2016df = month_dataframe(june_2016)
june_2017df = month_dataframe(june_2017)
frames1617 = (june_2016df, june_2017df)
june1617df = pd.concat(frames1617)
june1617df.sort_values("date", inplace = True)
june1617df.groupby("date").sum()

,precipitation,tobs
date,,
2016-06-01,4.34,495.0
2016-06-02,1.77,492.0
2016-06-03,0.77,504.0
2016-06-04,0.02,282.0
2016-06-05,0.99,362.0
2016-06-06,1.52,450.0
2016-06-07,2.09,527.0
2016-06-08,0.01,526.0
2016-06-09,2.48,507.0


In [20]:
# Identify key statistical data in June across all of the stations and years
june1617df.describe()

,precipitation,tobs
count,344.000000,385.000000
mean,0.166424,76.189610
std,0.281631,3.215019
min,0.000000,64.000000
25%,0.000000,74.000000
50%,0.040000,77.000000
75%,0.210000,79.000000
max,1.700000,83.000000


In [24]:
#Repeat the exercise for 2016 only - Dec 
#There is no data for 2017 Dec so this is for 2016 Dec only
dec_2016df.describe()

,precipitation,tobs
count,178.000000,200.000000
mean,0.199494,71.130000
std,0.342967,3.419292
min,0.000000,60.000000
25%,0.020000,69.000000
50%,0.060000,71.000000
75%,0.247500,74.000000
max,2.370000,78.000000
